In [98]:
import numpy as np
import pandas as pd
import matplotlib as plot
import math

# Pokemon Go Meta Analysis Notebook
This notebook is a one-stop data viewer for a Pokemon's Raid Counter and PvP Rankings in an effort to making storage clearing more easy.  

First, load in the Great League csv from PvPoke.com.

In [99]:
great_league = pd.read_csv('cp1500_all_overall_rankings.csv')
great_league.rename(columns={'Score': 'GL Score'}, inplace=True)
great_league = great_league[['Pokemon', 'GL Score']]
ultra_league = pd.read_csv('cp2500_all_overall_rankings.csv')
ultra_league.rename(columns={'Score': 'UL Score'}, inplace=True)
ultra_league = ultra_league[['Pokemon', 'UL Score']]
little_league = pd.read_csv('custom_cp500_all_custom_rankings.csv')
little_league.rename(columns={'Score': 'LC Score'}, inplace=True)
little_league = little_league[['Pokemon', 'LC Score']]
master_league = pd.read_csv('cp10000_all_overall_rankings.csv')
master_league.rename(columns={'Score': 'ML Score'}, inplace=True)
master_league = master_league[['Pokemon', 'ML Score']]

Now load in the Ultra League Rankings.

Now upload the Little League Rankings. *Note: this is subject to change based on current little league rules.*

Finally, load in Master League Rankings.

# Combining PvP Rankings

In [100]:
PvP_Rankings = pd.merge(great_league, ultra_league, on='Pokemon', how='outer')
PvP_Rankings = pd.merge(PvP_Rankings, master_league, on='Pokemon', how='outer')
PvP_Rankings = pd.merge(PvP_Rankings, little_league, on='Pokemon', how='outer')
PvP_Rankings = PvP_Rankings[['Pokemon', 'LC Score', 'GL Score', 'UL Score', 'ML Score']]

In [101]:
PvP_Rankings['GL Rank'] = PvP_Rankings['GL Score'].rank(ascending=False)
PvP_Rankings['UL Rank'] = PvP_Rankings['UL Score'].rank(ascending=False)
PvP_Rankings['ML Rank'] = PvP_Rankings['ML Score'].rank(ascending=False)
PvP_Rankings['LC Rank'] = PvP_Rankings['LC Score'].rank(ascending=False)
display(PvP_Rankings)

,Pokemon,LC Score,GL Score,UL Score,ML Score,GL Rank,UL Rank,ML Rank,LC Rank
0,Carbink,NaN,96.0,64.4,NaN,1.0,493.0,NaN,NaN
1,Registeel,NaN,94.4,98.9,75.9,2.0,1.0,95.0,NaN
2,Stunfisk (Galarian),NaN,93.8,89.4,60.0,3.0,28.0,254.0,NaN
3,Swampert (Shadow),NaN,93.3,93.2,83.9,4.0,7.0,48.0,NaN
4,Swampert,NaN,93.1,92.9,86.2,5.0,8.5,37.5,NaN
...,...,...,...,...,...,...,...,...,...
1138,Abra (Shadow),19.2,NaN,NaN,NaN,NaN,NaN,NaN,373.0
1139,Abra,19.0,NaN,NaN,NaN,NaN,NaN,NaN,374.0
1140,Cranidos,18.8,NaN,NaN,NaN,NaN,NaN,NaN,375.0
1141,Magikarp,5.6,NaN,NaN,NaN,NaN,NaN,NaN,377.0


# Loading in Raid Attacker Data

In [102]:
counters = pd.read_csv('comprehensive_dps.csv')
display(counters)

,Pokemon,Fast Move,Charged Move,DPS,TDO,ER,CP
0,Mega Rayquaza,Dragon Tail,Dragon Ascent,29.698,1122.2,73.63,5713
1,Mega Rayquaza,Air Slash,Dragon Ascent,28.503,1077.0,70.67,5713
2,Mega Mewtwo Y,Confusion,Psystrike,26.510,914.2,64.24,5610
3,Mega Mewtwo Y,Psycho Cut,Psystrike,26.374,909.5,63.91,5610
4,Mega Mewtwo X,Confusion,Psystrike,25.664,885.0,62.19,5429
...,...,...,...,...,...,...,...
14439,Shadow Shuckle,Rock Throw,Frustration,0.724,17.2,1.60,405
14440,Shadow Magikarp,Splash,Frustration,0.591,3.4,0.92,274
14441,Lycanroc (Dusk Form),Rock Throw,Crush Claw,0.369,7.8,0.79,2678
14442,Lycanroc (Dusk Form),Sucker Punch,Crush Claw,0.369,7.8,0.79,2678


In [103]:
counters = counters[['Pokemon', 'ER', 'DPS']]
display(counters)


,Pokemon,ER,DPS
0,Mega Rayquaza,73.63,29.698
1,Mega Rayquaza,70.67,28.503
2,Mega Mewtwo Y,64.24,26.510
3,Mega Mewtwo Y,63.91,26.374
4,Mega Mewtwo X,62.19,25.664
...,...,...,...
14439,Shadow Shuckle,1.60,0.724
14440,Shadow Magikarp,0.92,0.591
14441,Lycanroc (Dusk Form),0.79,0.369
14442,Lycanroc (Dusk Form),0.79,0.369


In [104]:
Master_Rankings = pd.merge(PvP_Rankings, counters, on='Pokemon', how='outer')
Master_Rankings = Master_Rankings[['Pokemon', 'LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'ER']]
display(Master_Rankings)

,Pokemon,LC Rank,GL Rank,UL Rank,ML Rank,ER
0,Carbink,NaN,1.0,493.0,NaN,15.54
1,Carbink,NaN,1.0,493.0,NaN,15.17
2,Carbink,NaN,1.0,493.0,NaN,14.89
3,Carbink,NaN,1.0,493.0,NaN,14.44
4,Carbink,NaN,1.0,493.0,NaN,14.05
...,...,...,...,...,...,...
14827,Cosmog,NaN,NaN,NaN,NaN,3.01
14828,Cosmoem,NaN,NaN,NaN,NaN,3.48
14829,Ditto,NaN,NaN,NaN,NaN,2.75
14830,Shadow Magikarp,NaN,NaN,NaN,NaN,2.07


In [105]:
Master_Rankings = Master_Rankings.groupby('Pokemon')[['LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'ER']].mean()
Master_Rankings['Raid Rank'] = Master_Rankings['ER'].rank(ascending=False)
Master_Rankings['Name'] = Master_Rankings.index
Final_Rankings = Master_Rankings[['Name', 'LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'Raid Rank']]
display(Final_Rankings)

,Name,LC Rank,GL Rank,UL Rank,ML Rank,Raid Rank
Pokemon,,,,,,
Flabébé,Flabébé,NaN,NaN,NaN,NaN,1067.0
5th Anniversary Pikachu,5th Anniversary Pikachu,NaN,NaN,NaN,NaN,1013.5
Abomasnow,Abomasnow,NaN,84.5,47.0,214.0,435.0
Abomasnow (Shadow),Abomasnow (Shadow),NaN,48.0,36.0,217.0,NaN
Abra,Abra,374.0,NaN,NaN,NaN,821.0
...,...,...,...,...,...,...
Zubat (Shadow),Zubat (Shadow),42.0,NaN,NaN,NaN,NaN
Zweilous,Zweilous,NaN,62.5,392.5,NaN,684.0
Zygarde (10% Forme),Zygarde (10% Forme),NaN,556.0,407.5,457.0,468.0


# Combining the Mega and Shadow Stats

In [106]:
def standardize_name(name):
    name = name.replace(' (Mega)', '').replace(' (Shadow)', '').strip()
    return name

# Apply standardization to both datasets
Final_Rankings['Standard_Name'] = Final_Rankings['Name'].apply(standardize_name)

# Update "Name" column with Shadow/Mega prefixes only for specific cases
for index, row in Final_Rankings.iterrows():
    if " (Alolan) (Shadow)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Shadow Alolan {row['Name']}"
    elif " (Shadow)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Shadow {row['Name']}"
    elif " (Mega)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Mega {row['Name']}"
    elif " (Hisuian)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Hisuian {row['Name']}"
    elif " (Galarian)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Galarian {row['Name']}"
    elif " (Alolan)" in row['Name']:
        Final_Rankings.at[index, 'Name'] = f"Alolan {row['Name']}"

display(Final_Rankings)

,Name,LC Rank,GL Rank,UL Rank,ML Rank,Raid Rank,Standard_Name
Pokemon,,,,,,,
Flabébé,Flabébé,NaN,NaN,NaN,NaN,1067.0,Flabébé
5th Anniversary Pikachu,5th Anniversary Pikachu,NaN,NaN,NaN,NaN,1013.5,5th Anniversary Pikachu
Abomasnow,Abomasnow,NaN,84.5,47.0,214.0,435.0,Abomasnow
Abomasnow (Shadow),Shadow Abomasnow (Shadow),NaN,48.0,36.0,217.0,NaN,Abomasnow
Abra,Abra,374.0,NaN,NaN,NaN,821.0,Abra
...,...,...,...,...,...,...,...
Zubat (Shadow),Shadow Zubat (Shadow),42.0,NaN,NaN,NaN,NaN,Zubat
Zweilous,Zweilous,NaN,62.5,392.5,NaN,684.0,Zweilous
Zygarde (10% Forme),Zygarde (10% Forme),NaN,556.0,407.5,457.0,468.0,Zygarde (10% Forme)


In [107]:
Final_Rankings['Name'] = Final_Rankings['Name'].str.replace(r'\s*\(Mega\)$|\s*\(Shadow\)$|\s*\(Galarian\)$|\s*\(Hisuian\)$|\s*\(Alolan\)|\s*\(Alolan\)\s*|\s*\(Shadow\)\s*$', '', regex=True)
Final_Rankings = Final_Rankings.groupby('Name')[['LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'Raid Rank']].mean().reset_index()
#Final_Rankings[['LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'Raid Rank']] = Final_Rankings[['LC Rank', 'GL Rank', 'UL Rank', 'ML Rank', 'Raid Rank']].apply(lambda x: math.ceil(x))
display(Final_Rankings)


,Name,LC Rank,GL Rank,UL Rank,ML Rank,Raid Rank
0,Flabébé,NaN,NaN,NaN,NaN,1067.0
1,5th Anniversary Pikachu,NaN,NaN,NaN,NaN,1013.5
2,Abomasnow,NaN,84.5,47.0,214.0,435.0
3,Abra,374.0,NaN,NaN,NaN,821.0
4,Absol,NaN,736.0,553.0,439.0,327.0
...,...,...,...,...,...,...
1404,Zubat,36.0,NaN,NaN,NaN,1270.0
1405,Zweilous,NaN,62.5,392.5,NaN,684.0
1406,Zygarde (10% Forme),NaN,556.0,407.5,457.0,468.0
1407,Zygarde (50% Forme),NaN,202.5,142.5,46.0,258.0
